In [1]:
import numpy as np
import pandas as pd
import math
import pylab
import scipy
import scipy.signal
from collections import OrderedDict
import time
from scipy.integrate import dblquad
import matplotlib.pyplot as plt
from matplotlib.pylab import *

# load MNIST data

In [2]:
import os, struct
from array import array as pyarray
from numpy import append, array, int8, uint8, zeros

def load_mnist(dataset="training", digits=np.arange(10),
               path=r'C:\Users\David\Documents\ETHZ 2015-2017\'16 HERBST\THESIS\MNIST'):
    """
    Loads MNIST files into 3D numpy arrays

    Adapted from: http://abel.ee.ucla.edu/cvxopt/_downloads/mnist.py
    """

    if dataset == "training":
        fname_img = os.path.join(path, 'train-images.idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels.idx1-ubyte')
    elif dataset == "testing":
        fname_img = os.path.join(path, 't10k-images.idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels.idx1-ubyte')
    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    flbl = open(fname_lbl, 'rb')
    magic_nr, size = struct.unpack(">II", flbl.read(8))
    lbl = pyarray("b", flbl.read())
    flbl.close()

    fimg = open(fname_img, 'rb')
    magic_nr, size, rows, cols = struct.unpack(">IIII", fimg.read(16))
    img = pyarray("B", fimg.read())
    fimg.close()

    ind = [ k for k in range(size) if lbl[k] in digits ]
    N = len(ind)

    images = zeros((N, rows, cols), dtype=uint8)
    labels = zeros((N, 1), dtype=int8)
    for i in range(len(ind)):
        images[i] = array(img[ ind[i]*rows*cols : (ind[i]+1)*rows*cols ]).reshape((rows, cols))
        labels[i] = lbl[ind[i]]

    return images, labels

In [3]:
images_train, labels_train = load_mnist(dataset="training")
images_test, labels_test = load_mnist(dataset="testing")

# scattering transform

In [4]:
def convolution2D(in1, in2, subsample=1):
    raw_out = scipy.signal.convolve2d(in1, in2, mode='full', boundary='fill', fillvalue=0)
    
    # trim so that output has desired dimensions (assume in1 is image, in2 is filter)
    shape = np.shape(in1)
    trim_size_x = np.floor(shape[0] / 2)
    trim_size_y = np.floor(shape[1] / 2)
    trimmed_out = raw_out[trim_size_x:-trim_size_x, trim_size_y:-trim_size_y]
    
    # subsample the trimmed output
    out = trimmed_out[::subsample, ::subsample].copy()
    
    return out

In [5]:
def complex_modulus(a):
    return np.sqrt(a.real**2 + a.imag**2)

In [7]:
def morlet_wavelet(u, scale, angle, sigma=0.8):
    assert(len(u) == 2)
    u = rotation_matrix(angle, radians=False).dot(u)
    u = u / scale
    c_sigma = (1 + np.exp(-sigma**2) - 2*np.exp(-0.75*sigma**2))**(-0.5)
    k_sigma = np.exp(-0.5*sigma**2)
    return c_sigma * (np.pi)**(-0.25) * np.exp(-0.5*np.linalg.norm(u)**2) \
        * (np.exp(sum(u)*sigma*1j) - k_sigma)

In [11]:
def rotation_matrix(angle, radians=False):
    if not radians:
        angle = angle * np.pi / 180
    return np.array( [[math.cos(angle),-math.sin(angle)], [math.sin(angle), math.cos(angle)]] )

In [36]:
# precompute and store wavelets!
def precompute_all_wvlts(scales, angles, shape=[28,28]):
    all_wvlts = {}
    for scale in scales:
        for angle in angles:
            this_wvlt = np.empty(shape, dtype=complex)
            for i in range(shape[0]):
                for j in range(shape[1]):
                    this_wvlt[i,j] = morlet_wavelet(np.array([i-(shape[0]-1)/2, j-(shape[1]-1)/2]),
                                                    scale, angle)
                    all_wvlts['(' + str(scale) + ',' + str(angle) + ')'] = this_wvlt
    return all_wvlts

In [29]:
def gaussian_window(u, sigma=1):
    return np.exp(-0.5 * np.linalg.norm(u) / sigma**2)

In [27]:
def windowed_scattering(image, window_size, window_type='Gaussian', alpha=1):
    shape = np.shape(image)
    window = np.empty(shape)
    
    if window_type == 'Gaussian':
        for i in range(shape[0]):
            for j in range(shape[1]):
                window[i,j] = gaussian_window(np.array([i-(shape[0]-1)/2, j-(shape[1]-1)/2]))
    else:
        raise("Error: invalid window_type!")
    
    # subsample at intervals window_size
    return convolution2D(image, window, alpha*2**window_size)

In [28]:
def produce_all_paths(scales=[1,2], angles=[0,15,30,45,60,75,90,105,120,135,150,165], depth=2):
    all_paths = OrderedDict()
    
    for i in range(depth):
        all_paths[i] = []
        
        if i == 0:
            # first layer
            for scale in scales:
                for angle in angles:
                    all_paths[i] += ['(' + str(scale) + ',' + str(angle) + ')']
        else:
            # start from last layer
            for path in all_paths[i-1]:
                steps = path.split('.')
                for scale in scales:
                    # frequency decreasing
                    if scale < eval(steps[-1])[0]:
                        for angle in angles:
                            all_paths[i] += [path + '.(' + str(scale) + ',' + str(angle) + ')']
    return all_paths

# scattering convolution network

In [37]:
def SCN(image, all_paths, all_wvlts, window_size, alpha=1, window_type='Gaussian',
        verbose_scattering_coeffs=False, pooling_type='complex'):
    
    U_p = {}
    U_p[''] = image
    if verbose_scattering_coeffs:
        output = {}
        output[''] = windowed_scattering(image, window_size, window_type, alpha)
    else:
        output = []
        output = np.append(output, windowed_scattering(image, window_size, window_type, alpha).flatten())
    
    # sort by layer so we can build upon previous layers
    for depth, paths in all_paths.items():
        for path in paths:

            # individual steps in a path
            steps = path.split('.')

            # previous layers
            path_minus_one = '.'.join(steps[:-1])
            use_prev = U_p[str(path_minus_one)]

            # current layer
            curr = eval(steps[-1])
            scale = curr[0]
            angle = curr[1]

            # use precomputed wavelets!
            use_wvlt = all_wvlts['(' + str(scale) + ',' + str(angle) + ')']

            # convolve previous and current layers
            convolved = convolution2D(use_prev, use_wvlt, 1) #alpha*2**scale)

            # store wavelet coeffs
            U_p[path] = complex_modulus(convolved)

            # store output scattering coeffs
            if verbose_scattering_coeffs:
                output[path] = windowed_scattering(U_p[path], window_size, window_type, alpha)
            else:
                output = np.append(output, windowed_scattering(U_p[path], window_size, window_type,
                                                               alpha).flatten())
    
    return output

In [38]:
def run_SCN(images, window_size, scales=[1,2], angles=[0,15,30,45,60,75,90,105,120,135,150,165],
            depth=2, alpha=1, window_type='Gaussian'):
    
    all_paths = produce_all_paths(scales, angles, depth)
    assert(type(all_paths) == OrderedDict)
    
    shape = np.shape(images[0])
    all_wvlts = precompute_all_wvlts(scales, angles, shape)
    
    out = []
    i = 0
    t0 = time.time()
    
    for image in images:
        SCN_coeffs = SCN(image=image, all_paths=all_paths, all_wvlts=all_wvlts,
                         window_size=window_size, alpha=alpha, window_type=window_type,
                         verbose_scattering_coeffs=False, pooling_type='complex')

        out.append(SCN_coeffs)
        
        t1 = time.time()
        
        i += 1
        
        if i % 100 == 0:
            print('100 images up to index ' + str(i) + ' took: ' + str(t1-t0) + ' secs!')
            t0 = time.time()
    
    return out

### testing SCN

In [26]:
imgs_5000 = images_train[:5000]
scattering_vecs_5000_sigma08 = run_SCN(imgs_5000, window_size=3, alpha=1)
np.savetxt('scattering_vecs_5000_sigma08.txt', np.array(scattering_vecs_5000_sigma08), delimiter=',')

100 images up to index 100 took: 1395.2124772071838 secs!
100 images up to index 200 took: 1371.9537830352783 secs!
100 images up to index 300 took: 1368.1965279579163 secs!
100 images up to index 400 took: 1551.9847929477692 secs!
100 images up to index 500 took: 1380.903100013733 secs!
100 images up to index 600 took: 1397.2863008975983 secs!
100 images up to index 700 took: 1390.7863328456879 secs!
100 images up to index 800 took: 1374.1138648986816 secs!
100 images up to index 900 took: 1359.5696799755096 secs!
100 images up to index 1000 took: 1377.270260810852 secs!
100 images up to index 1100 took: 1441.1661019325256 secs!
100 images up to index 1200 took: 1431.9936261177063 secs!
100 images up to index 1300 took: 1448.6228580474854 secs!
100 images up to index 1400 took: 1392.0996580123901 secs!
100 images up to index 1500 took: 1379.445240020752 secs!
100 images up to index 1600 took: 1400.9124190807343 secs!
100 images up to index 1700 took: 1497.1386420726776 secs!
100 image

In [18]:
imgs_5000_6000 = images_train[5000:6000]
scattering_vecs_5000_6000_sigma08 = run_SCN(imgs_5000_6000, window_size=3, alpha=1)
np.savetxt('scattering_vecs_5000_6000_sigma08.txt', np.array(scattering_vecs_5000_6000_sigma08),
           delimiter=',')

100 images up to index 100 took: 1579.0815739631653 secs!
100 images up to index 200 took: 1563.7993190288544 secs!
100 images up to index 300 took: 1361.8172109127045 secs!
100 images up to index 400 took: 1400.644005060196 secs!
100 images up to index 500 took: 1341.5281009674072 secs!
100 images up to index 600 took: 1357.610934972763 secs!
100 images up to index 700 took: 1342.2015690803528 secs!
100 images up to index 800 took: 1344.2649228572845 secs!
100 images up to index 900 took: 1346.840627193451 secs!
100 images up to index 1000 took: 1349.0115299224854 secs!


# generate variation MNIST and coeffs

In [36]:
def rand_transform(image):
    noise = np.random.uniform(0,255,(28,28))
    image = image + noise
    
    def cutoff_255(a):
        return min(a, 255)
    f = np.vectorize(cutoff_255)
    
    return f(image)

In [21]:
test_rand = rand_transform(orig_imgs_6000[0])
pylab.imshow(test_rand, cmap=pylab.cm.gray)
pylab.show()

In [62]:
def rot_transform(image):
    angle = 360 * np.random.random_sample()
    image = scipy.ndimage.interpolation.rotate(image, angle, reshape=False)
    return image

In [71]:
test_rot = rot_transform(orig_imgs_6000[0])
pylab.imshow(test_rot, cmap=pylab.cm.gray)
pylab.show()

In [68]:
rand_imgs_6000 = []
rot_imgs_6000 = []
for img in orig_imgs_6000:
    rand_imgs_6000 += [rand_transform(img)]
    rot_imgs_6000 += [rot_transform(img)]

In [69]:
rand_imgs_6000 = np.array(rand_imgs_6000).flatten().reshape(6000, 28, 28)
rot_imgs_6000 = np.array(rot_imgs_6000).flatten().reshape(6000, 28, 28)
np.savetxt('rand_imgs_6000.txt', rand_imgs_6000.reshape(6000,784), delimiter=',')
np.savetxt('rot_imgs_6000.txt', rot_imgs_6000.reshape(6000,784), delimiter=',')

### generate SCN coeffs

In [78]:
rand_scattering_vecs_6000 = run_SCN(rand_imgs_6000, window_size=3, alpha=1)
np.savetxt('rand_scattering_vecs_6000.txt', np.array(rand_scattering_vecs_6000), delimiter=',')

100 images up to index 100 took: 1108.8046650886536 secs!
100 images up to index 200 took: 1259.133581161499 secs!
100 images up to index 300 took: 1284.2534251213074 secs!
100 images up to index 400 took: 1133.0228040218353 secs!
100 images up to index 500 took: 1183.2477841377258 secs!
100 images up to index 600 took: 1262.8646838665009 secs!
100 images up to index 700 took: 1346.3921749591827 secs!
100 images up to index 800 took: 1076.3713901042938 secs!
100 images up to index 900 took: 1106.682126045227 secs!
100 images up to index 1000 took: 1145.7803449630737 secs!
100 images up to index 1100 took: 1543.8658900260925 secs!
100 images up to index 1200 took: 1554.5107798576355 secs!
100 images up to index 1300 took: 1533.1439199447632 secs!
100 images up to index 1400 took: 1523.428188085556 secs!
100 images up to index 1500 took: 1528.9704468250275 secs!
100 images up to index 1600 took: 1526.075630903244 secs!
100 images up to index 1700 took: 1480.9701437950134 secs!
100 images

In [79]:
rot_scattering_vecs_6000 = run_SCN(rot_imgs_6000, window_size=3, alpha=1)
np.savetxt('rot_scattering_vecs_6000.txt', np.array(rot_scattering_vecs_6000), delimiter=',')

100 images up to index 100 took: 1131.8400030136108 secs!
100 images up to index 200 took: 1070.149099111557 secs!
100 images up to index 300 took: 1139.514242887497 secs!
100 images up to index 400 took: 1068.753890991211 secs!
100 images up to index 500 took: 2064.4166259765625 secs!
100 images up to index 600 took: 1533.1503789424896 secs!
100 images up to index 700 took: 1212.8181800842285 secs!
100 images up to index 800 took: 1066.1626811027527 secs!
100 images up to index 900 took: 779.3297641277313 secs!
100 images up to index 1000 took: 1495.8728609085083 secs!
100 images up to index 1100 took: 972.600583076477 secs!
100 images up to index 1200 took: 528.6944859027863 secs!
100 images up to index 1300 took: 536.7726781368256 secs!
100 images up to index 1400 took: 509.10783100128174 secs!
100 images up to index 1500 took: 537.4291200637817 secs!
100 images up to index 1600 took: 525.6928341388702 secs!
100 images up to index 1700 took: 542.8258640766144 secs!
100 images up to 

# visuals

In [6]:
scattering_vecs = np.loadtxt('scattering_vecs_5000_sigma08.txt', delimiter=',')

In [6]:
scattering_vecs_rand = np.loadtxt('rand_scattering_vecs_6000.txt', delimiter=',')

In [15]:
scattering_vecs_rot = np.loadtxt('rot_scattering_vecs_6000.txt', delimiter=',')

In [19]:
# zeros, ones, fives
for index in (1,21,56,63,69, 3,6,23,24,59, 0,11,35,47,65):
    layer0 = layer0_SCNet_coeffs(index=index)
    pylab.imshow(layer0, cmap='gray', interpolation='nearest')
    pylab.show()
    plot_SCNet_coeffs(index=index, layer=1)
    plot_SCNet_coeffs(index=index, layer=2)

In [24]:
# nines
for index in (48, 788):
    layer0 = layer0_SCNet_coeffs(index=index)
    pylab.imshow(layer0, cmap='gray', interpolation='nearest')
    pylab.show()
    plot_SCNet_coeffs(index=index, layer=1)
    plot_SCNet_coeffs(index=index, layer=2)

In [7]:
# what does directly taking the window function look like
def layer0_SCNet_coeffs(index=0, scattering_vecs=scattering_vecs):
    
    vec = scattering_vecs[index][:16]
    
    return np.reshape(vec, (4,4))

In [16]:
def layer1_SCNet_coeffs(index=0, scattering_vecs=scattering_vecs):
    
    vec = scattering_vecs[index][16:400]
    vec1 = []
    
    for i in range(24):
        vec1 += [np.mean(vec[16*i:16*(i+1)])]
    
    mean_s1 = np.mean(vec1[:12])
    mean_s2 = np.mean(vec1[12:])
    mean = np.mean(vec1)
    
    use1 = vec1[:12] / max(vec1[:12])
    use2 = vec1[12:] / max(vec1[12:])
    
    out = np.append([0], use1)
    out = np.append(out, [0])
    out = np.append(out, use2)
    out = np.append(out, np.repeat([np.mean([np.mean(use1), np.mean(use2)])], 13))
    
    return np.reshape(out, (3,13))

In [17]:
def layer2_SCNet_coeffs(index=0, scattering_vecs=scattering_vecs):
    
    vec = scattering_vecs[index][400:]
    vec2 = []
    
    for i in range(144):
        vec2 += [np.mean(vec[16*i:16*(i+1)])]
    
    vec2 = vec2 / max(vec2)
    mean = np.mean(vec2)
    
    out = np.append([mean], vec2[:12])
    out = np.append(out, [mean])
    out = np.append(out, vec2[12:24])
    out = np.append(out, [mean])
    out = np.append(out, vec2[24:36])
    out = np.append(out, [mean])
    out = np.append(out, vec2[36:48])
    out = np.append(out, [mean])
    out = np.append(out, vec2[48:60])
    out = np.append(out, [mean])
    out = np.append(out, vec2[60:72])
    out = np.append(out, [mean])
    out = np.append(out, vec2[72:84])
    out = np.append(out, [mean])
    out = np.append(out, vec2[84:96])
    out = np.append(out, [mean])
    out = np.append(out, vec2[96:108])
    out = np.append(out, [mean])
    out = np.append(out, vec2[108:120])
    out = np.append(out, [mean])
    out = np.append(out, vec2[120:132])
    out = np.append(out, [mean])
    out = np.append(out, vec2[132:])
    out = np.append(out, np.repeat([mean], 13))
    
    return np.reshape(out, (13,13))

In [18]:
def plot_SCNet_coeffs(index=0, layer=1):
    
    if layer == 1:
        # values to plot
        c = layer1_SCNet_coeffs(index)
        
        # define grid
        th = array([pi/6 * n for n in range(13)])
        r = array(range(3))

    elif layer == 2:
        # values to plot
        c = layer2_SCNet_coeffs(index)
        
        # define grid
        th = array([pi/6 * n for n in range(13)])
        r = array(range(13))
    
    ax = subplot(111, projection='polar')
    
    # The smoothing
    TH = cbook.simple_linear_interpolation(th, 10)

    # padding C
    C = zeros((r.size, TH.size))
    oldfill = 0
    TH_ = TH.tolist()

    for i in range(th.size):
        fillto = TH_.index(th[i])

        for j, x in enumerate(c[:,i]):
            C[j, oldfill:fillto].fill(x)

        oldfill = fillto

    # The plotting
    th, r = meshgrid(TH, r)
    ax.pcolormesh(th, r, C)
    show()